# Part 1

You come upon a very unusual sight; a group of programs here appear to be dancing.

There are sixteen programs in total, named a through p. They start by standing in a line: a stands in position 0, b stands in position 1, and so on until p, which stands in position 15.

The programs' dance consists of a sequence of dance moves:

- Spin, written sX, makes X programs move from the end to the front, but maintain their order otherwise. (For example, s3 on abcde produces cdeab).
- Exchange, written xA/B, makes the programs at positions A and B swap places.
- Partner, written pA/B, makes the programs named A and B swap places.

For example, with only five programs standing in a line (abcde), they could do the following dance:

- s1, a spin of size 1: eabcd.
- x3/4, swapping the last two programs: eabdc.
- pe/b, swapping programs e and b: baedc.

After finishing their dance, the programs end up in order baedc.

You watch the dance for a while and record their dance moves (your puzzle input). In what order are the programs standing after their dance?

In [1]:
import string

In [2]:
n_to_a = {i: c for i, c in enumerate(string.ascii_lowercase)}

In [3]:
def puzzle_input():
    a_to_n = {c: i for i, c in enumerate(string.ascii_lowercase)}
    moves = []
    with open('./inputs/day16/input.txt') as f:
        for move in f.read().split(','):
            if move[0] == 's':
                moves.append(('s', int(move[1:])))
            elif move[0] == 'x':
                moves.append(('x', tuple(int(x) for x in move[1:].split('/'))))
            elif move[0] == 'p':
                moves.append(('p', tuple(a_to_n[c] for c in move[1:].split('/'))))
    return moves

In [4]:
puzzle_input()[:5]

[('x', (11, 4)), ('p', (3, 7)), ('x', (10, 5)), ('s', 3), ('x', (0, 7))]

In [5]:
%load_ext cython

In [6]:
%%cython
cpdef void dance(int num, list array, list moves):
    cdef int spin, pivot, pos1, pos2, c1, c2
    for move in moves:
        if move[0] == 's':
            spin = move[1]
            pivot = num - spin
            array[:spin], array[spin:] = array[pivot:], array[:pivot]
        elif move[0] == 'x':
            pos1, pos2 = move[1]
            array[pos1], array[pos2] = array[pos2], array[pos1]
        elif move[0] == 'p':
            c1, c2 = move[1]
            pos_map = {val: pos for pos, val in enumerate(array)}
            array[pos_map[c2]], array[pos_map[c1]] = c1, c2

In [7]:
num = 5
array = list(range(num))
dance(num, array, [('s', 1), ('x', (3, 4)), ('p', (4, 1))])
result = ''.join(n_to_a[i] for i in array)
assert result == 'baedc', result

In [8]:
num = 16
input_ = puzzle_input()
array = list(range(num))
dance(num, array, input_)
''.join(n_to_a[i] for i in array)

'ehdpincaogkblmfj'

Answer: `ehdpincaogkblmfj`

In [9]:
%timeit dance(num, array, input_)

4.1 ms ± 388 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Part 2

Now that you're starting to get a feel for the dance moves, you turn your attention to the dance as a whole.

Keeping the positions they ended up in from their previous dance, the programs perform it again and again: including the first dance, a total of one billion (1000000000) times.

In the example above, their second dance would begin with the order baedc, and use the same dance moves:

- s1, a spin of size 1: cbaed.
- x3/4, swapping the last two programs: cbade.
- pe/b, swapping programs e and b: ceadb.

In what order are the programs standing after their billion dances?

In [10]:
num = 5
array = list(range(num))
for _ in range(2):
    dance(num, array, [('s', 1), ('x', (3, 4)), ('p', (4, 1))])
result = ''.join(n_to_a[i] for i in array)
assert result == 'ceadb', result

A billion loops is too many, need a trick.
Check for cycles in the state letters that would allow us to bypass all that iteration.

In [11]:
%%time
num = 16
input_ = puzzle_input()
array = list(range(num))
seen = {tuple(array): 1}
for _ in range(100):
    dance(num, array, input_)
    if tuple(array) in seen:
        print(_ + 1, array)
        break
    else:
        seen[tuple(array)] = _ + 1

48 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
CPU times: user 238 ms, sys: 4.94 ms, total: 243 ms
Wall time: 242 ms


In [12]:
1000000000 % 48

16

In [13]:
result = list(filter(lambda t: t[1] == 16, seen.items()))

In [14]:
''.join(n_to_a[i] for i in result[0][0])

'bpcekomfgjdlinha'